In [1]:
import tensorflow as tf
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt

/home/pavithran/anaconda3/envs/ten1.4/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/pavithran/anaconda3/envs/ten1.4/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/pavithran/anaconda3/envs/ten1.4/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/pavithran/anaconda3/envs/ten1.4

In [2]:
def read_images(dataset_path):
    global classes
    im_path,labels=list(),list()
    classes=os.listdir(dataset_path)
    for i in classes:
        path=os.path.join(dataset_path,i)
        for j in os.listdir(path):
            path1=os.path.join(path,j)
            im_path.append(path1)
            lab=classes.index(i)
            if lab==0:
                labels.append([0,1])
            else:
                labels.append([1,0])
    for i in range(len(classes)):
        print('{}:{}'.format(classes[i],i))
    im_path=tf.convert_to_tensor(im_path)
    labels=tf.convert_to_tensor(labels)
    labels=tf.cast(labels,tf.float32)
    image,labels=tf.train.slice_input_producer([im_path,labels],shuffle=True)
    image = tf.read_file(image)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize_images(image, [32, 32])
    # Normalize
    image = image * 1.0/127.5 - 1.0
    X, Y = tf.train.batch([image, labels], batch_size=32,capacity=32 * 8,num_threads=4)
    return X,Y

In [3]:
dataset_path='dataset/'
X,Y=read_images(dataset_path)

sos:0
normal:1
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(tuple(tensor_list)).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.range(limit).shuffle(limit).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(input_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensors(tensor).repeat(num_epochs)`.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions fo

In [4]:
def getModel(x):
    with tf.variable_scope('conv1') as scope:
        w=tf.get_variable('w',shape=[3,3,3,32],initializer=tf.contrib.layers.xavier_initializer())
        b=tf.get_variable('b',initializer=tf.zeros([32]))
        l1=tf.nn.conv2d(x,w,strides=[1,1,1,1],padding='VALID')
        l1_b=tf.nn.bias_add(l1,b)
        l1_act=tf.nn.relu(l1_b)
    print(l1_act)
    with tf.variable_scope('conv2') as scope:
        w=tf.get_variable('w',shape=[3,3,32,64],initializer=tf.contrib.layers.xavier_initializer())
        b=tf.get_variable('b',initializer=tf.zeros([64]))
        l2=tf.nn.conv2d(l1_act,w,strides=[1,1,1,1],padding='VALID')
        l2_b=tf.nn.bias_add(l2,b)
        l2_act=tf.nn.relu(l2_b)
    l3_max=tf.nn.max_pool(l2_act,[1,2,2,1],strides=[1,2,2,1],padding='VALID')
    print(l3_max)
    with tf.variable_scope('dense1') as scope:
        w=tf.get_variable('w',shape=[14*14*64,120],initializer=tf.contrib.layers.xavier_initializer())
        b=tf.get_variable('b',initializer=tf.zeros([120]))
        l4=tf.reshape(l2_act,[-1,14*14*64])
        l4=tf.matmul(l4,w)
        l4_b=tf.nn.bias_add(l4,b)
        l4_act=tf.nn.relu(l4_b)
    print(l4_act)

In [5]:
getModel(X)

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Tensor("conv1/Relu:0", shape=(32, 30, 30, 32), dtype=float32)
Tensor("MaxPool:0", shape=(32, 14, 14, 64), dtype=float32)
Tensor("dense1/Relu:0", shape=(128, 120), dtype=float32)
